In [1]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2
import playsound
from playsound import*
import numpy as np
import json
import time
import winsound

In [2]:
detector = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
recognizer = cv2.face.LBPHFaceRecognizer_create()
f = open("driver_names.json", "r")
driver_dict = json.load(f)
f.close()
#driver_dict = {"1": "tony", "2":"steve", "3":"thor", "4": "natasha"}
print(driver_dict)
cam_in = cv2.VideoCapture(0)
recognizer.read("trained21.yml")# please insert address here.
while 1:
    ret, img = cam_in.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = detector.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h),(255,255,0),2)
        id, conf = recognizer.predict(gray[y:y+h,x:x+w])
        if str(id) in driver_dict:
            name = driver_dict[str(id)]
        else:
            name = "Access Denied"
        cv2.putText(img, name, (x, y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    #winsound.PlaySound(path, winsound.SND_ASYNC)
    #playsound(path)
    cv2.imshow("Face",img)
    #time.sleep(15)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cam_in.release()
cv2.destroyAllWindows()

{'1': 'IronMan'}


In [3]:
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [4]:
# thresh = 0.25
# frame_check = 20
thresh = 0.3
frame_check = 30
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [6]:
cap=cv2.VideoCapture(0)
f=0
while (True):
    ret, frame=cap.read()
    frame = imutils.resize(frame, width=450)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    subjects = detect(gray, 0)
    for subject in subjects:
        shape = predict(gray, subject)
        shape = face_utils.shape_to_np(shape)#converting to NumPy Array
        leftEye = shape[lStart:lEnd]
        rightEye = shape[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        ear = (leftEAR + rightEAR) / 2.0
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        if (ear < thresh):
            f += 1
            #print (flag)
            if (f >= frame_check):
                cv2.putText(frame, "ALERT!", (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                cv2.putText(frame, "ALERT!", (10,325),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                playsound('ALARM.mp3')
            break
        else:
            f = 0
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if( key == ord("q")):
        cv2.destroyAllWindows()
        cap.release()
        break